In [1]:
import clingo

In [2]:
inst1 = """

node(1).
node(2).
node(3).

edge((1,2)).
edge((1,3)).
edge((2,1)).
edge((2,3)).
edge((3,1)).
edge((3,2)).

edge_weight((1,2), 10).
edge_weight((2,3), 10).
edge_weight((3,1), 10).
edge_weight((2,1), 100).
edge_weight((3,2), 100).
edge_weight((1,3), 100).

mintime(1).
maxtime(3).

"""

In [3]:
reasoning = """

time(I..A) :- mintime(1), maxtime(A).

lastwalk(L) :- walk(_,L), not walk(_,L+1).

edgecardinality(C) :- C = #count { U,V : edge((U,V)), U < V }.

1 { walk((1,V),I) : edge((1,V)) } 1 :- mintime(I).
1 { walk((V,W),I+1) : edge((V,W)) } 1 :- walk((U,V), I), mintime(I).
1 { walk((V,W),T) : edge((V,W)), walk((_,V),T-1) } 1 :- T <= E, edgecardinality(E), walk(_,T-1), maxtime(A), T<=A.
0 { walk((V,W),T) : edge((V,W)), walk((_,V),T-1) } 1 :- T > E, edgecardinality(E), walk(_,T-1), maxtime(A), T<=A.

:- mintime(I), not walk((1,_), I).
:- lastwalk(L), walk((_,1), L).
:- edge((U,V)), edge((V,U)), not walk((U,V), _), not walk((V,U), _).



:- edge(), edge(E2) &

"""